In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential

plt.rcParams['font.family'] = 'Malgun Gothic'

# 전처리

### train 데이터 전처리

In [2]:
# 데이터 load
df = pd.read_csv('./한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
df

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105
...,...,...,...,...
368083,2018-12-31,20,H,681.033
368084,2018-12-31,21,H,669.961
368085,2018-12-31,22,H,657.941
368086,2018-12-31,23,H,610.953


In [3]:
# 시간 데이터 결합을 위해 0~23으로 변환
df['시간'] = df['시간'] - 1
df

,연월일,시간,구분,공급량
0,2013-01-01,0,A,2497.129
1,2013-01-01,1,A,2363.265
2,2013-01-01,2,A,2258.505
3,2013-01-01,3,A,2243.969
4,2013-01-01,4,A,2344.105
...,...,...,...,...
368083,2018-12-31,19,H,681.033
368084,2018-12-31,20,H,669.961
368085,2018-12-31,21,H,657.941
368086,2018-12-31,22,H,610.953


In [4]:
# 한자리 정수로 표현된 시간 데이터 앞에 0 추가
df['시간'] = np.where(df['시간'] == 0, '00', df['시간'])
df['시간'] = np.where(df['시간'] == '1', '01', df['시간'])
df['시간'] = np.where(df['시간'] == '2', '02', df['시간'])
df['시간'] = np.where(df['시간'] == '3', '03', df['시간'])
df['시간'] = np.where(df['시간'] == '4', '04', df['시간'])
df['시간'] = np.where(df['시간'] == '5', '05', df['시간'])
df['시간'] = np.where(df['시간'] == '6', '06', df['시간'])
df['시간'] = np.where(df['시간'] == '7', '07', df['시간'])
df['시간'] = np.where(df['시간'] == '8', '08', df['시간'])
df['시간'] = np.where(df['시간'] == '9', '09', df['시간'])
df

,연월일,시간,구분,공급량
0,2013-01-01,00,A,2497.129
1,2013-01-01,01,A,2363.265
2,2013-01-01,02,A,2258.505
3,2013-01-01,03,A,2243.969
4,2013-01-01,04,A,2344.105
...,...,...,...,...
368083,2018-12-31,19,H,681.033
368084,2018-12-31,20,H,669.961
368085,2018-12-31,21,H,657.941
368086,2018-12-31,22,H,610.953


In [5]:
# 시점 데이터 생성
df['시점'] = df['연월일'].map(str) + ' ' + df['시간'].map(str)
df

,연월일,시간,구분,공급량,시점
0,2013-01-01,00,A,2497.129,2013-01-01 00
1,2013-01-01,01,A,2363.265,2013-01-01 01
2,2013-01-01,02,A,2258.505,2013-01-01 02
3,2013-01-01,03,A,2243.969,2013-01-01 03
4,2013-01-01,04,A,2344.105,2013-01-01 04
...,...,...,...,...,...
368083,2018-12-31,19,H,681.033,2018-12-31 19
368084,2018-12-31,20,H,669.961,2018-12-31 20
368085,2018-12-31,21,H,657.941,2018-12-31 21
368086,2018-12-31,22,H,610.953,2018-12-31 22


In [6]:
# 시점 데이터 형 변환
df['시점'] = pd.to_datetime(df['시점'], format='%Y-%m-%d %H')
df

,연월일,시간,구분,공급량,시점
0,2013-01-01,00,A,2497.129,2013-01-01 00:00:00
1,2013-01-01,01,A,2363.265,2013-01-01 01:00:00
2,2013-01-01,02,A,2258.505,2013-01-01 02:00:00
3,2013-01-01,03,A,2243.969,2013-01-01 03:00:00
4,2013-01-01,04,A,2344.105,2013-01-01 04:00:00
...,...,...,...,...,...
368083,2018-12-31,19,H,681.033,2018-12-31 19:00:00
368084,2018-12-31,20,H,669.961,2018-12-31 20:00:00
368085,2018-12-31,21,H,657.941,2018-12-31 21:00:00
368086,2018-12-31,22,H,610.953,2018-12-31 22:00:00


In [7]:
# 연월일, 시간 데이터 제거
df.drop(columns=['연월일', '시간'], inplace=True)
df

,구분,공급량,시점
0,A,2497.129,2013-01-01 00:00:00
1,A,2363.265,2013-01-01 01:00:00
2,A,2258.505,2013-01-01 02:00:00
3,A,2243.969,2013-01-01 03:00:00
4,A,2344.105,2013-01-01 04:00:00
...,...,...,...
368083,H,681.033,2018-12-31 19:00:00
368084,H,669.961,2018-12-31 20:00:00
368085,H,657.941,2018-12-31 21:00:00
368086,H,610.953,2018-12-31 22:00:00


In [8]:
# 시점 데이터 index로 변경
df.set_index('시점', inplace=True)
df

,구분,공급량
시점,,
2013-01-01 00:00:00,A,2497.129
2013-01-01 01:00:00,A,2363.265
2013-01-01 02:00:00,A,2258.505
2013-01-01 03:00:00,A,2243.969
2013-01-01 04:00:00,A,2344.105
...,...,...
2018-12-31 19:00:00,H,681.033
2018-12-31 20:00:00,H,669.961
2018-12-31 21:00:00,H,657.941


### test 데이터 전처리

In [9]:
# test 데이터 load
test = pd.read_csv('data/test.csv')
test

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A
...,...
15115,2019-03-31 20 H
15116,2019-03-31 21 H
15117,2019-03-31 22 H
15118,2019-03-31 23 H


In [10]:
# 일자, 시간, 구분 분리
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]
test

,일자|시간|구분,일자,시간,구분
0,2019-01-01 01 A,2019-01-01,1,A
1,2019-01-01 02 A,2019-01-01,2,A
2,2019-01-01 03 A,2019-01-01,3,A
3,2019-01-01 04 A,2019-01-01,4,A
4,2019-01-01 05 A,2019-01-01,5,A
...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,H
15116,2019-03-31 21 H,2019-03-31,21,H
15117,2019-03-31 22 H,2019-03-31,22,H
15118,2019-03-31 23 H,2019-03-31,23,H


In [11]:
# 원래 column 제거
test.drop(columns='일자|시간|구분', inplace=True)
test

,일자,시간,구분
0,2019-01-01,1,A
1,2019-01-01,2,A
2,2019-01-01,3,A
3,2019-01-01,4,A
4,2019-01-01,5,A
...,...,...,...
15115,2019-03-31,20,H
15116,2019-03-31,21,H
15117,2019-03-31,22,H
15118,2019-03-31,23,H


In [12]:
# 시간 데이터 결합을 위해 0~23으로 변환
test['시간'] = test['시간'] - 1
test

,일자,시간,구분
0,2019-01-01,0,A
1,2019-01-01,1,A
2,2019-01-01,2,A
3,2019-01-01,3,A
4,2019-01-01,4,A
...,...,...,...
15115,2019-03-31,19,H
15116,2019-03-31,20,H
15117,2019-03-31,21,H
15118,2019-03-31,22,H


In [13]:
# 한자리 정수로 표현된 시간 데이터 앞에 0 추가
test['시간'] = np.where(test['시간'] == 0, '00', test['시간'])
test['시간'] = np.where(test['시간'] == '1', '01', test['시간'])
test['시간'] = np.where(test['시간'] == '2', '02', test['시간'])
test['시간'] = np.where(test['시간'] == '3', '03', test['시간'])
test['시간'] = np.where(test['시간'] == '4', '04', test['시간'])
test['시간'] = np.where(test['시간'] == '5', '05', test['시간'])
test['시간'] = np.where(test['시간'] == '6', '06', test['시간'])
test['시간'] = np.where(test['시간'] == '7', '07', test['시간'])
test['시간'] = np.where(test['시간'] == '8', '08', test['시간'])
test['시간'] = np.where(test['시간'] == '9', '09', test['시간'])
test

,일자,시간,구분
0,2019-01-01,00,A
1,2019-01-01,01,A
2,2019-01-01,02,A
3,2019-01-01,03,A
4,2019-01-01,04,A
...,...,...,...
15115,2019-03-31,19,H
15116,2019-03-31,20,H
15117,2019-03-31,21,H
15118,2019-03-31,22,H


In [14]:
# 시점 데이터 생성
test['시점'] = test['일자'].map(str) + ' ' + test['시간'].map(str)
test

,일자,시간,구분,시점
0,2019-01-01,00,A,2019-01-01 00
1,2019-01-01,01,A,2019-01-01 01
2,2019-01-01,02,A,2019-01-01 02
3,2019-01-01,03,A,2019-01-01 03
4,2019-01-01,04,A,2019-01-01 04
...,...,...,...,...
15115,2019-03-31,19,H,2019-03-31 19
15116,2019-03-31,20,H,2019-03-31 20
15117,2019-03-31,21,H,2019-03-31 21
15118,2019-03-31,22,H,2019-03-31 22


In [15]:
# 시점 데이터 형 변환
test['시점'] = pd.to_datetime(test['시점'], format='%Y-%m-%d %H')
test

,일자,시간,구분,시점
0,2019-01-01,00,A,2019-01-01 00:00:00
1,2019-01-01,01,A,2019-01-01 01:00:00
2,2019-01-01,02,A,2019-01-01 02:00:00
3,2019-01-01,03,A,2019-01-01 03:00:00
4,2019-01-01,04,A,2019-01-01 04:00:00
...,...,...,...,...
15115,2019-03-31,19,H,2019-03-31 19:00:00
15116,2019-03-31,20,H,2019-03-31 20:00:00
15117,2019-03-31,21,H,2019-03-31 21:00:00
15118,2019-03-31,22,H,2019-03-31 22:00:00


In [16]:
# 일자, 시간 데이터 제거
test.drop(columns=['일자', '시간'], inplace=True)
test

,구분,시점
0,A,2019-01-01 00:00:00
1,A,2019-01-01 01:00:00
2,A,2019-01-01 02:00:00
3,A,2019-01-01 03:00:00
4,A,2019-01-01 04:00:00
...,...,...
15115,H,2019-03-31 19:00:00
15116,H,2019-03-31 20:00:00
15117,H,2019-03-31 21:00:00
15118,H,2019-03-31 22:00:00


In [18]:
# 시점 데이터 index로 변경
test.set_index('시점', inplace=True)
test

,구분
시점,
2019-01-01 00:00:00,A
2019-01-01 01:00:00,A
2019-01-01 02:00:00,A
2019-01-01 03:00:00,A
2019-01-01 04:00:00,A
...,...
2019-03-31 19:00:00,H
2019-03-31 20:00:00,H
2019-03-31 21:00:00,H


In [19]:
# 전처리 된 데이터 저장
df.to_csv('train.csv')
test.to_csv('test.csv')